In [9]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
df = pd.read_csv("data.csv")

df = df.iloc[:, 1:]

X = df.iloc[:, 0:-1]
y = df['label']

In [10]:
df

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,label
0,167,179,175,183,207,190,176,186,199,192,...,254,255,251,247,247,231,207,183,159,0
1,52,56,68,80,72,94,111,107,106,114,...,252,251,246,245,240,239,239,239,239,0
2,187,183,163,183,167,162,152,131,134,141,...,252,253,254,254,254,255,251,243,231,0
3,8,8,8,8,8,8,8,8,8,13,...,8,8,8,8,8,8,8,8,8,0
4,8,8,8,8,8,8,8,8,10,8,...,8,8,8,8,8,8,8,8,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,60,104,72,211,195,180,159,181,155,106,...,170,197,153,117,173,72,36,12,8,1
1756,108,116,108,92,80,67,82,112,159,196,...,9,58,128,133,116,175,207,155,56,1
1757,92,84,68,76,60,46,41,26,22,9,...,141,138,139,184,180,68,88,20,8,1
1758,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,1


In [11]:
def pipeline(frac):
    print("----------------------", frac,"----------------------------")
    from sklearn.model_selection import train_test_split
    df_train, df_test = train_test_split(df, test_size=0.25,random_state=42)

    df_train['Random_Mask'] = True 
    df_train.loc[df_train.sample(frac=frac, random_state=0).index, 'Random_Mask'] = False

    
    df_train['target']=df_train.apply(lambda x: x['label'] if x['Random_Mask']==False else -1, axis=1)

    
    df_labeled_data = df_train[ df_train['Random_Mask']==False ]
    df_unlabeled_data = df_train[ df_train['Random_Mask']==True ]


    X_ = df_train[df_train['Random_Mask']==False].iloc[:,list(range(0,4096))]
    Y_ = df_train[df_train['Random_Mask']==False]['target']

    x_ = df_test.iloc[:,list(range(0,4096))]
    y_ = df_test['label']

    from sklearn.svm import SVC

    svm = SVC(C =0.1, gamma= 0.0001, kernel='poly',probability=True, cache_size=200)
    svm.fit(X_,Y_)

    print('---------- SVM Model - Evaluation on Test Data ----------')
    print("Train Accuracy Score: ", svm.score(X_, Y_))
    print("Test Accuracy Score: ",accuracy_score(y_,svm.predict(x_)))
    print(classification_report(y_, svm.predict(x_)))
    



    from sklearn.semi_supervised import SelfTrainingClassifier

    X_ = df_train.iloc[:,list(range(0,4096))]
    Y_ = df_train['target']

    svm = SVC(C =0.1, gamma= 0.0001, kernel='poly',probability=True, cache_size=200)
    self_training_model = SelfTrainingClassifier(base_estimator=svm, 
                                                threshold=0.7, 
                                                criterion='threshold', 
                                                max_iter=100, 
                                                verbose=True 
                                                )

    clf_ST = self_training_model.fit(X_, Y_)

    print('---------- Self Training Model - Evaluation on Test Data ----------')
    print("Train Accuracy Score: ", clf_ST.score(X_, Y_))
    accuracy_score_ST = clf_ST.score(x_, y_)
    print('Test Accuracy Score: ', accuracy_score_ST)
    print(classification_report(y_, clf_ST.predict(x_)))
    print("-------------------------------------------------------------------")


In [12]:

###### -------------------- Diffrent Ratio of Labeled Data from 5% to 95% -------------------------

pipeline(0.05)

pipeline(0.1)
pipeline(0.15)

pipeline(0.2)
pipeline(0.25)

pipeline(0.3)
pipeline(0.35)

pipeline(0.4)
pipeline(0.45)

pipeline(0.5)
pipeline(0.55)

pipeline(0.6)
pipeline(0.65)

pipeline(0.7)
pipeline(0.75)

pipeline(0.8)
pipeline(0.85)

pipeline(0.9)
pipeline(0.95)

---------------------- 0.05 ----------------------------
---------- SVM Model - Evaluation on Test Data ----------
Train Accuracy Score:  1.0
Test Accuracy Score:  0.7977272727272727
              precision    recall  f1-score   support

           0       0.77      0.85      0.81       224
           1       0.83      0.74      0.78       216

    accuracy                           0.80       440
   macro avg       0.80      0.80      0.80       440
weighted avg       0.80      0.80      0.80       440

End of iteration 1, added 939 new labels.
End of iteration 2, added 241 new labels.
End of iteration 3, added 41 new labels.


KeyboardInterrupt: 